In [3]:
import numpy as np
import xgboost as xgb
from typing import Tuple

def relu(x):
    return max(x,0)

def focal_obj(Tw, Ti):
    def custom_obj(expected: np.ndarray, predt: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        rows = expected.shape[0]
        grad = np.zeros((rows, classes), dtype=float)
        hess = np.zeros((rows, classes), dtype=float)
        for r in range(predt.shape[0]):
            for c in range(predt.shape[1]):
                y = expected[r][c]
                y_hat = predt[r][c]
                g = 2*(y_hat - y) + relu(2*(y_hat - Tw)) + relu(2*(Ti - y_hat))
                h = 2 + relu(2*(y_hat - Tw)/abs(y_hat - Tw)) + relu(2*(Ti - y_hat)/abs(Ti - y_hat))
                grad[r, c] = g
                hess[r, c] = h
        grad = grad.reshape((rows * classes, 1))
        hess = hess.reshape((rows * classes, 1))
        return grad, hess
    return custom_obj
                                            

clf = xgb.XGBClassifier(tree_method="hist", objective=focal_obj)